# Instructions to run this script
1) It should be run in a computer with GPU. It is better to run it in Google Colab, if so, please create a shortcut of the shared folder in your drive.

2) Locate this file "people_counter.ipynb" in a folder.

3) Put your videos that you want to process in a folder.

4) Create a folder for output videos.

5) Mount Drive

6) Set your default path

7) Install the library "face_recognition"

8) Run "People Counter"


# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


# Set Default Directory

In [3]:
import os
!ls "drive/My Drive/people-counter/Maxplank"
os.chdir("drive/My Drive/people-counter/Maxplank")

face_detector.ipynb  input_videos  output_videos


# Install Libraries

In [6]:
!pip install face_recognition

# People Counter

In [7]:
# Import libraries
import glob
import os
import pandas as pd
import numpy as np
import cv2
import face_recognition

class Count_Person:
  #Initialize my class by giving the : 
    # 1) paths of input videos
    # 2) paths ofoutput videos 
    # 3) Name of cvs file where the name of each video and how many people contains
    # 4) Threshold to assure unique identity of a person recognition
    # 5) skip_frame to do the processing faster

  def __init__(self, dir_input, dir_output, output_file_name,  threshold, skip_frame):
    # Function to initialize class parameters
    self.dir_input = dir_input
    self.dir_output = dir_output
    self.output_file_name = output_file_name
    self.threshold = threshold
    self.skip_frame=skip_frame

  def retrieve_list_videos(self):
    #Function to retrieve a list of videos name stored in a given directory
    list_videos=os.listdir(self.dir_input)
    return list_videos

  def detect_count_people(self, myinput_video):
    #Function to return number of people in a given video
    output_movie=self.dir_output+myinput_video[:-4]+".avi"

    input_movie = cv2.VideoCapture(self.dir_input+myinput_video)
    length = int(input_movie.get(cv2.CAP_PROP_FRAME_COUNT))
    print("\t"+self.dir_input+myinput_video)
    # Initialize variables
    face_locations = []
    face_encodings = []
    all_face_encodings=[]
    face_names = []
    frame_number = 0
    writer = None
    while True:
      # Grab a single frame of video
      ret, frame = input_movie.read()
      frame_number += 1

      # Quit when the input video file ends
      if not ret:
          break
      if frame_number % self.skip_frame==0:
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        h, w, chan=frame.shape                  #Get current frame size
        frame=cv2.resize(frame, (w+500, h+500)) #Resize each frame
        rgb_frame = frame[:, :, ::-1]
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_frame, model="cnn")
        #face_locations = face_recognition.face_locations(rgb_frame, number_of_times_to_upsample=2, model="cnn")
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        # Loop through each face found in the unknown frame
        found=0
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
          if len(all_face_encodings)!=0:
            for ii in range(len(all_face_encodings)):
              face_distance = face_recognition.face_distance([all_face_encodings[ii]], face_encoding)
              #print(face_distance)
              if face_distance[0] <=self.threshold :  # faces with a distance of threshold (0.65) or less should be a match.
                found=1
                continue
          # Draw a box around the face
          cv2.rectangle(frame, (left-20, top-20), (right+20, bottom+20), (0, 0, 255), 2)

          # Draw a label with a name below the face
          #cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 0, 255), cv2.FILLED)
          #font = cv2.FONT_HERSHEY_DUPLEX
          #cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

          if found==0:
            all_face_encodings.append(face_encoding)     

        # Write the resulting image to the output video file
        if frame_number%100==0:
          print("\tWriting frame {} / {}".format(frame_number, length))
        (h, w) = rgb_frame.shape[:2]
        if output_movie is not None and writer is None:
          fourcc = cv2.VideoWriter_fourcc(*"MJPG")
          writer = cv2.VideoWriter(output_movie, fourcc, 30, (w, h), True)
        if writer is not None:
          writer.write(frame)
    # All done!
    input_movie.release()
    cv2.destroyAllWindows()
    #Change video format
    #!ffmpeg -i output_movie self.dir_output+myinput_video[:-4]+".mp4"
    no_people=len(all_face_encodings)
    print("\t[INFO] # unique person: {}".format(no_people))
    return (no_people)

  def create_csv_file(self, videos_list, list_no_people ):
    #Function recieve the list of videos and the q
    zippedList =  list(zip(videos_list, list_no_people))
    df=pd.DataFrame(zippedList, columns = ['Video' , 'No. of People'])
    df.index += 1 
    df.to_csv(self.dir_output+self.output_file_name)
  
  def print_cvs_file(self):
    #Function to print out the cvs content
    df=pd.read_csv(self.dir_output+self.output_file_name)
    print(df.head(50))

def main():
    print("Hello to people detection and counting!")
    #######  Initialize variables #######
    dir_input="input_videos/"
    dir_output="output_videos/"
    output_file_name="data.csv"
    threshold=0.65
    skip_frame=2
    list_no_people=list()
    ###### Initialize class variable ######
    cp=Count_Person(dir_input, dir_output, output_file_name,  threshold, skip_frame)
    ###### Retrieve the list of video in a given directory ######
    videoslist=cp.retrieve_list_videos()
    print("\n Here is the list of videos that I am going to process:")
    print(videoslist)
    print("\n ########## Processing "+str(len(videoslist))+" videos #########" )
    for myvideo in range(len(videoslist)):
      print("\n=========>> "+videoslist[myvideo]+" <<==========")
      no_people=cp.detect_count_people(videoslist[myvideo])
      list_no_people.append(no_people)
      
    ###### Write results to a CSV file  ######
    cp.create_csv_file(videoslist, list_no_people)

    ##### Print the content of the CSV file #####
    cp.print_cvs_file()

    print("Good bye!")
    
if __name__ == "__main__":
    main()

Hello to people detection and counting!

 Here is the list of videos that I am going to process:
['video2.mp4', 'video3.mp4', 'video1.mp4']

 ########## Processing 3 videos #########

=========>> video2.mp4 <<==========
	input_videos/video2.mp4
	Writing frame 100 / 2144
	Writing frame 200 / 2144
	Writing frame 300 / 2144
	Writing frame 400 / 2144
	Writing frame 500 / 2144
	Writing frame 600 / 2144
	Writing frame 700 / 2144
	Writing frame 800 / 2144
	Writing frame 900 / 2144
	Writing frame 1000 / 2144
	Writing frame 1100 / 2144
	Writing frame 1200 / 2144
	Writing frame 1300 / 2144
	Writing frame 1400 / 2144
	Writing frame 1500 / 2144
	Writing frame 1600 / 2144
	Writing frame 1700 / 2144
	Writing frame 1800 / 2144
	Writing frame 1900 / 2144
	Writing frame 2000 / 2144
	Writing frame 2100 / 2144
	[INFO] # unique person: 4

=========>> video3.mp4 <<==========
	input_videos/video3.mp4
	Writing frame 100 / 2199
	Writing frame 200 / 2199
	Writing frame 300 / 2199
	Writing frame 400 / 2199
	Wri